In [53]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [54]:
device = torch.device("cuda")

In [55]:
transform = transforms.Compose([
     transforms.Resize((224,224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [56]:
from sklearn.model_selection import train_test_split
data_path = '/kaggle/input/dl-dseb-coursework/data/train'

train_data, test_data = train_test_split(
    torchvision.datasets.ImageFolder(data_path, transform=transform),
    test_size=0.1,
    random_state=42
)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [57]:
#categories
root=pathlib.Path(data_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [58]:
print(classes)

['butterfly', 'cat', 'chicken', 'cow', 'dog', 'elephant', 'horse', 'sheep', 'spider', 'squirrel', 'train_map']


In [59]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet201', pretrained=False) 

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [60]:
num_epochs = 50
batch_size = 32

from torch import nn

model.fc = nn.Linear(2048,10, bias = True)
model= model.to(device)
# Loss and optimizer
optimizer=Adam(model.parameters(),lr=0.0001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

# Train the model
total_step = len(train_loader)

In [61]:
train_count = len(train_data)
train_count

7200

In [62]:
test_count = len(test_data)
test_count

800

In [63]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(2.3074) Train Accuracy: 0.3136111111111111 Test Accuracy: 0.3975
Epoch: 1 Train Loss: tensor(1.4899) Train Accuracy: 0.4708333333333333 Test Accuracy: 0.49375
Epoch: 2 Train Loss: tensor(1.2499) Train Accuracy: 0.57625 Test Accuracy: 0.565
Epoch: 3 Train Loss: tensor(1.0585) Train Accuracy: 0.6388888888888888 Test Accuracy: 0.60625
Epoch: 4 Train Loss: tensor(0.8983) Train Accuracy: 0.6936111111111111 Test Accuracy: 0.61125
Epoch: 5 Train Loss: tensor(0.7738) Train Accuracy: 0.7377777777777778 Test Accuracy: 0.62125
Epoch: 6 Train Loss: tensor(0.6577) Train Accuracy: 0.7723611111111112 Test Accuracy: 0.64125
Epoch: 7 Train Loss: tensor(0.5262) Train Accuracy: 0.8268055555555556 Test Accuracy: 0.57375
Epoch: 8 Train Loss: tensor(0.4434) Train Accuracy: 0.8504166666666667 Test Accuracy: 0.675
Epoch: 9 Train Loss: tensor(0.3562) Train Accuracy: 0.8838888888888888 Test Accuracy: 0.63375
Epoch: 10 Train Loss: tensor(0.2919) Train Accuracy: 0.9069444444444444 Test

In [64]:
train_path='/kaggle/input/dl-dseb-coursework/data/train'
pred_path= '/kaggle/input/dl-dseb-coursework/data/test'

In [65]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [66]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet201', pretrained=False) 
       
num_classes = 10
model.fc = torch.nn.Linear(2048, num_classes)
checkpoint = torch.load('best_checkpoint.model')
model.load_state_dict(checkpoint, strict=False)

model = model.to('cuda')
model.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [67]:
transform = transforms.Compose([
     transforms.Resize((224,224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [68]:
from PIL import Image

In [69]:
def prediction(img_path, transformer):
    image = Image.open(img_path).convert('RGB')  
    image_tensor = transformer(image).float()
    
    image_tensor = image_tensor.unsqueeze(0)
    
    if torch.cuda.is_available():
        image_tensor = image_tensor.to('cuda')
    
    input = Variable(image_tensor)

    output = model(input)
    
    index = output.data.cpu().numpy().argmax() 
    
    pred = classes[index]
    
    return pred

In [70]:
images_path=glob.glob(pred_path+'/*.jpg')

In [71]:
pred_dict = {}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]] = prediction(i, transform)

In [72]:
pred_dict

{'1269.jpg': 'cow',
 '623.jpg': 'butterfly',
 '764.jpg': 'butterfly',
 '1700.jpg': 'squirrel',
 '1786.jpg': 'squirrel',
 '1075.jpg': 'squirrel',
 '771.jpg': 'butterfly',
 '208.jpg': 'chicken',
 '820.jpg': 'chicken',
 '1789.jpg': 'dog',
 '473.jpg': 'elephant',
 '1792.jpg': 'squirrel',
 '1411.jpg': 'sheep',
 '1031.jpg': 'cat',
 '1833.jpg': 'spider',
 '1749.jpg': 'squirrel',
 '1501.jpg': 'sheep',
 '1995.jpg': 'spider',
 '1385.jpg': 'cow',
 '1971.jpg': 'spider',
 '1383.jpg': 'cow',
 '333.jpg': 'horse',
 '1354.jpg': 'cow',
 '1024.jpg': 'dog',
 '537.jpg': 'elephant',
 '45.jpg': 'cat',
 '369.jpg': 'horse',
 '1626.jpg': 'cat',
 '56.jpg': 'dog',
 '654.jpg': 'butterfly',
 '89.jpg': 'dog',
 '1711.jpg': 'dog',
 '20.jpg': 'dog',
 '1339.jpg': 'cow',
 '275.jpg': 'dog',
 '1161.jpg': 'cat',
 '785.jpg': 'butterfly',
 '1940.jpg': 'butterfly',
 '1777.jpg': 'dog',
 '1380.jpg': 'elephant',
 '212.jpg': 'dog',
 '239.jpg': 'elephant',
 '792.jpg': 'butterfly',
 '1009.jpg': 'cat',
 '1157.jpg': 'dog',
 '58.jpg': 

In [73]:
label_mapping = {
    "dog": 0,
    "horse": 1,
    "elephant": 2,
    "butterfly": 3,
    "chicken": 4,
    "cat": 5,
    "cow": 6,
    "sheep": 7,
    "squirrel": 8,
    "spider": 9
}

new_pred_dict = {}
for img_name, pred_label in pred_dict.items():
    if pred_label in label_mapping:
        new_label = label_mapping[pred_label]
        new_pred_dict[img_name] = new_label
    else:
        new_pred_dict[img_name] = pred_label

In [74]:
import pandas as pd
df = pd.DataFrame.from_dict(new_pred_dict, orient='index', columns=['Label'])
df['ID'] = df.index.str.split('/').str[-1].str.split('\\').str[-1] 
df = df[['ID', 'Label']]
df.to_csv('result.csv', index=False)

In [75]:
df.head()

,ID,Label
1269.jpg,1269.jpg,6
623.jpg,623.jpg,3
764.jpg,764.jpg,3
1700.jpg,1700.jpg,8
1786.jpg,1786.jpg,8
